In [3]:
# 🐍 Step 1: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models


In [4]:
# 📥 Step 2: Load Dataset
# Oxford-IIIT Pet comes with pre-labeled segmentation masks
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)
train_ds = dataset['train']
test_ds = dataset['test']


AssertionError: Failed to construct dataset "oxford_iiit_pet", builder_kwargs "{'version': '3.*.*', 'data_dir': None}": Dataset oxford_iiit_pet cannot be loaded at version 3.*.*, only: 4.0.0.

In [ ]:
# 🧹 Step 3: Preprocess Data
IMG_SIZE = 128

def normalize_img(datapoint):
    image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE)) / 255.0
    mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))
    mask = tf.where(mask == 2, 1, mask)  # pet (1), background (0)
    return image, tf.cast(mask, tf.float32)

train_data = train_ds.map(normalize_img).cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_ds.map(normalize_img).cache().batch(32).prefetch(tf.data.AUTOTUNE)



In [5]:
# 🧠 Step 4: Build U-Net Model
def unet_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D()(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, 2, strides=2, padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, 3, activation='relu', padding='same')(u5)

    u6 = layers.Conv2DTranspose(128, 2, strides=2, padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, 3, activation='relu', padding='same')(u6)

    u7 = layers.Conv2DTranspose(64, 2, strides=2, padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, 3, activation='relu', padding='same')(u7)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)
    return models.Model(inputs, outputs)


NameError: name 'IMG_SIZE' is not defined